In [ ]:
! pip install -q sentence-transformers torch tqdm

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/LaBSE")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [3]:
import json
import numpy as np
from tqdm import tqdm

def build_embeddings(
    jsonl_path,
    output_prefix,
    batch_size=16
):
    texts = []
    doc_ids = []

    # Read corpus
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for doc_id, line in enumerate(f):
            try:
                doc = json.loads(line)
            except:
                continue

            text = doc.get("body", "").strip()
            if text:
                texts.append(text)
                doc_ids.append(doc_id)

    print(f"Loaded {len(texts)} documents from {jsonl_path}")

    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        emb = model.encode(
            batch,
            normalize_embeddings=True,
            show_progress_bar=False
        )
        embeddings.append(emb)

    embeddings = np.vstack(embeddings)

    np.save(f"{output_prefix}_embeddings.npy", embeddings)
    with open(f"{output_prefix}_doc_ids.json", "w") as f:
        json.dump(doc_ids, f)

    print(f"Saved embeddings → {output_prefix}_embeddings.npy")
    print(f"Saved doc IDs → {output_prefix}_doc_ids.json")


In [5]:
# Bangla embeddings
build_embeddings(
    jsonl_path="bangla_corpus.jsonl",
    output_prefix="bangla"
)

# English embeddings
build_embeddings(
    jsonl_path="english_corpus.jsonl",
    output_prefix="english"
)


Loaded 5694 documents from bangla_corpus.jsonl


100%|██████████| 356/356 [01:18<00:00,  4.56it/s]


Saved embeddings → bangla_embeddings.npy
Saved doc IDs → bangla_doc_ids.json
Loaded 3855 documents from english_corpus.jsonl


100%|██████████| 241/241 [00:52<00:00,  4.60it/s]

Saved embeddings → english_embeddings.npy
Saved doc IDs → english_doc_ids.json
